In [ ]:
import os
import sys
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))


from autogen.agentchat.contrib.orchestrator_agent import OrchestratorAgent
from autogen.agentchat import AssistantAgent, UserProxyAgent
import os
import json
import logging

logging.basicConfig(level=logging.INFO)

config_list = [
    {
        "model": os.environ.get("MODEL"),
        "api_key": os.environ.get("API_KEY"),
        "base_url": os.environ.get("BASE_URL", None)
    }
]

llm_config = {
    "config_list": config_list,
    "temperature": 0,
    "timeout": 120
}

executor = UserProxyAgent(
    name="executor",
    llm_config=llm_config,
    max_consecutive_auto_reply=10,
    human_input_mode="NEVER",
    description="executor. execute the code written by the coder and report the result.",
    code_execution_config={
        "work_dir": "coding",
        "use_docker": False,
    }
)

coder = AssistantAgent(
    name="coder",
    llm_config=llm_config,
    system_message="You are a helpful AI coding assistant. You write clean, efficient code and explain your solutions clearly.",
    max_consecutive_auto_reply=10,
    human_input_mode="NEVER",
    description="coder. Write code to solve tasks.",
)


orchestrator = OrchestratorAgent(
    name="Orchestrator",
    llm_config=llm_config,
    agents=[executor,coder],
    max_consecutive_auto_reply=10,
    max_stalls_before_replan=2,  #3
    max_replans=1, #3 
    return_final_answer=True,
    description="An orchestrator that manages conversation flow and handles errors."
)

task = """write and execute a python prints the prime numbers between 1 and 1000"""

orchestrator.initiate_chat(message=task)
        

